In [1]:
import pandas as pd
import numpy as np
# load libraries
import sys
from e2eml.regression import regression_blueprints
from e2eml.full_processing import postprocessing
from sklearn.model_selection import train_test_split
import gc

[nltk_data] Downloading package punkt to /home/thomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/thomas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/thomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
target = "score"

In [3]:
#comments = pd.read_csv("jigsaw_toxic_comments_to_score.csv")
#comments

In [4]:
df_train = pd.read_csv("jigsaw_toxic_validation_data.csv")
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can't even give a consistent argument - is the opening only supposed to mention significant aspects, or the """"most significant"""" ones? \n\n""",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is a horrible thing you put on my talk page. 128.61.19.93,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for prostitution ring. - Crunch Captain.",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.,0,0,0,0,0,0


In [5]:
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].mean(axis=1)

min_len = (df_train['score'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['score'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments
df_train = pd.concat([df_train[df_train['score'] > 0], df_y0_undersample])  # make new df

df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,0.32,1.5,0.16,0.0,0.64,0.0,0.436667
12,0005c987bdfc9d4b,"Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...",0.32,0.0,0.00,0.0,0.00,0.0,0.053333
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming back! Tosser.",0.32,0.0,0.00,0.0,0.00,0.0,0.053333
42,001810bf8c45bf5f,"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...\n\n1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!\n\n2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!\n\n3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!\n\nBeware of the Dark Side!",0.32,0.0,0.16,0.0,0.64,1.5,0.436667
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",0.32,0.0,0.16,0.0,0.64,0.0,0.186667
...,...,...,...,...,...,...,...,...,...
137462,df957e1303ef8c68,"""\n\nRead more\nYou can read more by going to anomtalk.com except with """"n"""" instead of """"m"""".""",0.00,0.0,0.00,0.0,0.00,0.0,0.000000
82635,dd0b9714b24c3c66,Sounds like a deal. Thanks for hearing me out.,0.00,0.0,0.00,0.0,0.00,0.0,0.000000
149206,58e756534c67c8e7,"Bone to pick \n\nWhat makes you the rulemaker on all things bears related? Bad and Good go into articles. Get a life and stay off wikipedia for a good month or two. People like you are the reason that wikipedia is a joke. Too much of your personal feelings goes into making sure nothing you don't agree with gets put into an article. Last I checked your job wasn't encyclopedia writer, so let others ideas be put into articles.214.13.192.187",0.00,0.0,0.00,0.0,0.00,0.0,0.000000
87807,eade5e302541c3a9,"""\n\n Semi-vegetarianism section \n\nI like Flyer22's intention in that one's recent edits, though I find the particulars a little unclear. Here is my proposal for the section ( ):\n===Semi-vegetarian diets===\nSemi-vegetarian diets primarily consist of vegetarian foods, though occasional exceptions are made for some non-vegetarian foods, including fish, poultry and even red meat. These diets may be followed by those who choose to reduce the amount of animal flesh consumed as a way of transitioning to a vegetarian diet, or for health, environmental, or other reasons. The term """"semi-vegetarian"""" is contested by most vegetarian groups, who believe that vegetarianism must exclude all animal flesh. Many individuals describe themselves as """"vegetarian"""" while practicing a semi-vegetarian diet. Semi-vegetarian diets include: \n Flexitarianism: A diet that consists primarily of vegetarian food, but includes occasional exceptions such as red meat.\n Pescetarianism: A diet that is mainly ...",0.00,0.0,0.00,0.0,0.00,0.0,0.000000


In [6]:
df_train = df_train.sample(frac=1, random_state=5000)
train_df = df_train.head(25000).copy()
val_df = df_train.tail(5000).copy()

In [7]:
val_df_target = val_df[target]
val_df = val_df.drop(target, axis=1)
val_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
92199,f67e6b569638f366,have no life coo \noco coco haaaaaaaaaaaaz,0.32,0.0,0.00,0.0,0.00,0.0
84107,e10b2247952b0913,"a fire, you worthless, friendless bastard. I hate you.",0.32,0.0,0.16,0.0,0.64,0.0
26362,45cc66085311b07b,":Internet media in Shreveport\nA tag has been placed on Internet media in Shreveport, requesting that it be speedily deleted from Wikipedia. This has been done because it is an article about a certain web site, blog, forum, or other community of web users that does not assert the importance or significance of that web location. Please read our criteria for speedy deletion, particularly item 7 under Articles, as well as notability guidelines for websites. Please note that articles must be on notable subjects and should provide references to reliable sources which verify their content.\n\nPlease do not remove the speedy deletion tag yourself. To contest the tagging and request that administrators wait a while for you to add contextual material, please affix the template {{hangon}} to the page, and put a note on Talk:Internet media in Shreveport. Feel free to leave a note on my talk page if you have any questions about this.",0.00,0.0,0.00,0.0,0.00,0.0
56013,95aa6f9a0a4cbf0c,Bin Laden and his 19 so-called hijackers were patsies.,0.32,0.0,0.00,0.0,0.00,0.0
3131,0872804fee5e6408,"""\n\n Re: AIP \n\nGo fuck yourself. -The'FortyFive' """,0.32,0.0,0.16,0.0,0.64,0.0
...,...,...,...,...,...,...,...,...
120607,852589ee2f697076,"COULD YOU JUST LISTEN TO ME FOR 5 SECONDS?!! I know you hate me, and I know you want to leave me alone, but it makes me sad that just for one minor edit, you just block me. I know that's what you have to do. By the way, this is NOT vandalism. Truth. I will leave you alone forever if you give me one more chance. PLEASE!",0.00,0.0,0.00,0.0,0.00,0.0
142797,fbac8e568e73e7e6,"""\nThank you. | Talk """,0.00,0.0,0.00,0.0,0.00,0.0
32417,5640714285ce656c,"""\n\n Why LLD (Mont)? \n\nIn the infobox, Trudeau is titled """"LLD (Mont)"""" which presumably means that he got his LLD from University of Montreal. Is it standard for all Canadian LLDs to have their alma mater specified in parentheses? """,0.00,0.0,0.00,0.0,0.00,0.0
14590,2687391618b89683,WE ARE GOING TO MAKE SURE YOU ARE EXPOSED!! DROP YOUR RELATIONS WITH ANTI AMERICAN HATE GROUPS 'WE ARE GOING TO MAKE SURE YOU ARE EXPOSED!! DROP YOUR RELATIONS WITH ANTI AMERICAN HATE GROUPS,0.32,0.0,0.00,0.0,0.00,0.0


In [8]:
toxic_ml = regression_blueprints.RegressionBluePrint(datasource=df_train, 
                        target_variable=target,
                        train_split_type='cross',
                        preferred_training_mode='auto',
                        nlp_transformer_columns='comment_text',
                        transformer_chosen='neoyipeng/twitter-roberta-base-sentiment-mlm',
                        transformer_epochs=3,
                        ml_task='regression')
# cardiffnlp/twitter-roberta-base-dec2021
# neoyipeng/twitter-roberta-base-sentiment-mlm

toxic_ml.blueprint_step_selection_nlp_transformers["random_synonym_replacement"] = True

toxic_ml.transformer_settings["train_batch_size"] = 16
toxic_ml.transformer_settings["test_batch_size"] = 16
toxic_ml.transformer_settings["pred_batch_size"] = 1

Ml task is regression
Preferred training mode auto has been chosen. e2eml will automatically detect, if LGBM and Xgboost can use GPU acceleration and optimize the workflow accordingly.


In [9]:
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer()

Started Execute test train split at 06:48:25.
Started Handle rare features at 06:48:43.
Started Sort columns alphabetically at 06:48:44.


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             55165 non-null  object 
 2   identity_hate  55165 non-null  float64
 3   insult         55165 non-null  float64
 4   obscene        55165 non-null  float64
 5   score          55165 non-null  float64
 6   severe_toxic   55165 non-null  float64
 7   threat         55165 non-null  float64
 8   toxic          55165 non-null  float64
 9   kfold          55165 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 4.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             55165 non-null  object 
 2   identi

Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

---------------0start-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             55165 non-null  object 
 2   identity_hate  55165 non-null  float64
 3   insult         55165 non-null  float64
 4   obscene        55165 non-null  float64
 5   score          55165 non-null  float64
 6   severe_toxic   55165 non-null  float64
 7   threat         55165 non-null  float64
 8   toxic          55165 non-null  float64
 9   kfold          55165 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 4.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             55165 non-null  object 
 2   identity_hate  55165 non-null  float64
 3   insult         55165 non-null  float64
 4   obscene        55165 non-null  float64
 5   score          55165 non-null  float64
 6   severe_toxic   55165 non-null  float64
 7   threat         55165 non-null  float64
 8   toxic          55165 non-null  float64
 9   kfold          55165 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 4.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55165 entries, 0 to 55164
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   55165 non-null  object 
 1   id             55165 non-null  object 
 2   identi

Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

['/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model4.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model0.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model3.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model2.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model1.pth']


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

The R2 score is 0.8015999870374634
The MAE score is 0.03421169612311453
The Median absolute error score is 0.013849972883860279
The MSE score is 0.06085460346750533
The RMSE score is 0.003703282763187312


In [10]:
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
import gc
# Save pipeline
save_to_production(toxic_ml, file_name='toxic_ml_bert_instance')

# Predict on external set

In [11]:
external_df = pd.read_csv('Toxic_FinalBalancedDataset.csv')
external_df = external_df.rename(columns={'tweet': 'comment_text'})
external_df

,Unnamed: 0,Toxicity,comment_text
0,0,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,1,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,4,0,factsguide: society now #motivation
...,...,...,...
56740,56740,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
56741,56741,1,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
56742,56742,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
56743,56743,1,youu got wild bitches tellin you lies


In [12]:
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer(external_df)

Started Execute test train split at 09:46:03.
Started Handle rare features at 09:46:03.
Started Sort columns alphabetically at 09:46:03.


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

['/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model4.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model0.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model3.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model2.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model1.pth']


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

In [13]:
external_df[target] = toxic_ml.predicted_values["nlp_transformer"]

In [14]:
external_df.to_csv('Toxic_FinalBalancedDataset_labeled.csv')

# Concat both datasets

In [21]:
full_train_df = pd.concat([df_train[['comment_text', target]], external_df[['comment_text', target]]])
full_train_df = full_train_df.sample(frac=1, random_state=5000)

# Final training

In [22]:
try:
    del toxic_ml
    _ = gc.collect()
except Exception:
    pass

In [23]:
toxic_ml = regression_blueprints.RegressionBluePrint(datasource=full_train_df, 
                        target_variable=target,
                        train_split_type='cross',
                        preferred_training_mode='auto',
                        nlp_transformer_columns='comment_text',
                        transformer_chosen='neoyipeng/twitter-roberta-base-sentiment-mlm',
                        transformer_epochs=3,
                        ml_task='regression')
# cardiffnlp/twitter-roberta-base-dec2021
# neoyipeng/twitter-roberta-base-sentiment-mlm

toxic_ml.blueprint_step_selection_nlp_transformers["random_synonym_replacement"] = False

toxic_ml.transformer_settings["train_batch_size"] = 16
toxic_ml.transformer_settings["test_batch_size"] = 16
toxic_ml.transformer_settings["pred_batch_size"] = 1

Ml task is regression
Preferred training mode auto has been chosen. e2eml will automatically detect, if LGBM and Xgboost can use GPU acceleration and optimize the workflow accordingly.


In [24]:
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer()

Started Execute test train split at 10:49:47.
Started Handle rare features at 10:49:47.
Started Sort columns alphabetically at 10:49:48.


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
 3   bins          89195 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 2.7+ MB
None


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

---------------0start-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
 3   bins          89195 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 2.7+ MB
None


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89195 entries, 0 to 89194
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   comment_text  89195 non-null  object 
 1   score         89195 non-null  float64
 2   kfold         89195 non-null  int64  
 3   bins          89195 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 2.7+ MB
None


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

['/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model4.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model0.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model3.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model2.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model1.pth']


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

The R2 score is 0.8853722694530894
The MAE score is 0.01853425811463225
The Median absolute error score is 0.005521809682250023
The MSE score is 0.040096064895874625
The RMSE score is 0.0016076944201341894


In [25]:
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
import gc
# Save pipeline
save_to_production(toxic_ml, file_name='toxic_ml_bert_instance')